In [65]:
!pip install torch torchquantum

     |████████████████████████████████| 890.2 MB 3.2 kB/s s eta 0:00:01    |██████▏                         | 171.9 MB 98.1 MB/s eta 0:00:08
  Using cached torchquantum-0.1.4-py3-none-any.whl (119 kB)
     |████████████████████████████████| 21.0 MB 87.9 MB/s eta 0:00:01
     |████████████████████████████████| 849 kB 85.9 MB/s eta 0:00:01
     |████████████████████████████████| 557.1 MB 5.6 kB/s s eta 0:00:01     |███████████████████████████▍    | 476.2 MB 106.8 MB/s eta 0:00:01
     |████████████████████████████████| 317.1 MB 16 kB/s /s eta 0:00:01
  Using cached qiskit-0.32.1-py3-none-any.whl
  Using cached pylatexenc-2.10-py3-none-any.whl
  Using cached torchpack-0.3.1-py3-none-any.whl (34 kB)
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
     |████████████████████████████████| 24.3 MB 55.3 MB/s eta 0:00:01
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux201

In [7]:
!pip install qiskit pennylane-qiskit

  Using cached qiskit-0.38.0-py3-none-any.whl
  Using cached PennyLane_qiskit-0.24.0-py3-none-any.whl (34 kB)
  Using cached qiskit_aer-0.11.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.2 MB)
  Using cached qiskit_terra-0.21.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.7 MB)
  Using cached qiskit_ibmq_provider-0.19.2-py3-none-any.whl (240 kB)
  Using cached websockets-10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (112 kB)
  Using cached websocket_client-1.4.1-py3-none-any.whl (55 kB)
  Using cached requests_ntlm-1.1.0-py2.py3-none-any.whl (5.7 kB)
  Using cached sympy-1.10.1-py3-none-any.whl (6.4 MB)
  Using cached tweedledum-1.1.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (943 kB)
  Using cached stevedore-3.5.0-py3-none-any.whl (49 kB)
  Using cached shared_memory38-0.1.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (25 kB)
  Using cached symengine-0.9.2-cp37-cp37m-man

In [95]:
from importlib import reload

import datasets
import create_gate_circs
import train_circ

reload(datasets)
reload(create_gate_circs)
reload(train_circ)

<module 'train_circ' from '/root/train_circ.py'>

In [ ]:
from datasets import TorchDataset
from create_gate_circs import get_circ_params, TQCirc, generate_random_gate_circ
from train_circ import train_tq_model

import pickle as pkl
import torch
import os
import numpy as np

num_qubits = 10

device = torch.device('cpu')

dataset = 'mnist_10'

train_data = TorchDataset(dataset, 'angle', 1, True)
test_data = TorchDataset(dataset, 'angle', 1, False)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1024, sampler=torch.utils.data.RandomSampler(train_data))

val_test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=100, sampler=torch.utils.data.SequentialSampler(test_data))

loss = torch.nn.functional.nll_loss
val_loss = torch.nn.NLLLoss(reduction='none')

for i in range(366, 400):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)

    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, True).to(device)
    opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
    losses_list = []
    accs_list = []
    
    for j in range(1):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        accs_history = train_tq_model(model, opt, loss, train_data_loader, test_data_loader, 120000, 1000, 1000)
        
#         curr_val_losses = []
#         curr_acc = 0
        
#         for k in range(100):
#             val_x, val_y = next(iter(val_test_data_loader))

#             val_preds = model(val_x)
#             curr_val_losses.append(val_loss(val_preds, val_y.to(torch.long)).detach().numpy().flatten())
        
#             val_class = torch.argmax(val_preds.detach(), 1).detach().numpy().flatten()

#             curr_acc += np.mean(val_class == val_y.detach().numpy().flatten())
        
#         np.save(curr_train_dir + '/accs_history_{}.npy'.format(j + 1), accs_history)
        
#         losses_list.append(curr_val_losses)
#         accs_list.append(curr_acc / 100)
        
#         print(curr_acc)
        
    torch.save(model.state_dict(), curr_dir + '/model.pt')
#     np.save(curr_dir + '/val_losses.npy', losses_list)
#     np.savetxt(curr_dir + '/accs.txt', accs_list)

Step 1 | Accuracy: 0.0634765625
Step 1001 | Accuracy: 0.255859375
Step 2001 | Accuracy: 0.2548828125
Step 3001 | Accuracy: 0.25390625
Step 4001 | Accuracy: 0.2734375
Step 5001 | Accuracy: 0.240234375
Step 6001 | Accuracy: 0.2734375
Step 7001 | Accuracy: 0.3017578125
Step 8001 | Accuracy: 0.2900390625
Step 9001 | Accuracy: 0.2978515625
Step 10001 | Accuracy: 0.28125
Step 11001 | Accuracy: 0.279296875
Step 12001 | Accuracy: 0.302734375
Step 13001 | Accuracy: 0.29296875
Step 14001 | Accuracy: 0.2861328125
Step 15001 | Accuracy: 0.2822265625
Step 16001 | Accuracy: 0.26953125
Step 17001 | Accuracy: 0.298828125
Step 18001 | Accuracy: 0.291015625
Step 19001 | Accuracy: 0.291015625
Step 20001 | Accuracy: 0.2822265625
Step 21001 | Accuracy: 0.3037109375
Step 22001 | Accuracy: 0.3017578125
Step 23001 | Accuracy: 0.298828125
Step 24001 | Accuracy: 0.3349609375
Step 25001 | Accuracy: 0.271484375
Step 26001 | Accuracy: 0.310546875
Step 27001 | Accuracy: 0.3125
Step 28001 | Accuracy: 0.32421875
Step

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'fmnist_4'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

num_qubits = 4
num_embeds = 16
num_params = 18

device_name = 'ibmq_lima'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(800, 1000):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

#     if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):
    if True:
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
#             acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

ibmqfactory.load_account:WARNING:2022-10-24 04:07:23,017: Credentials are already in use. The existing account in the session will be replaced.


0.9322265363526508 1.2713957877828326 801
0.9009165057346822 1.4352208235133395 802
1.7071067811865477 1.611803631029316 803


In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'vowel_2'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './experiment_data/{}/trained_circuits'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

num_cdcs = 128
num_shots = 1024

device_name = 'ibmq_manila'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 

    noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 
    
    noise_metric_dir = circ_dir + '/noise_metric'
    
    if not os.path.exists(noise_metric_dir):
        os.mkdir(noise_metric_dir)
    
    device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

    if not os.path.exists(device_noise_metric_dir):
        os.mkdir(device_noise_metric_dir)
    
    params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
    batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]
    
    noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
    noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

    actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))
    
    tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)
    
    np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
    np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])
    
    score_tvds.append(1 - tvd)
    actual_tvds.append(actual_tvd)
    
    print(1 - tvd, actual_tvd, i)

# np.savetxt(curr_dir + '/noise_scores.txt', score_tvds)
# np.savetxt(curr_dir + '/noisy_fids.txt', actual_tvds)

ibmqfactory.load_account:WARNING:2022-11-05 17:26:08,928: Credentials are already in use. The existing account in the session will be replaced.


0.8645477294921875 0.873443603515625 0
0.8762969970703125 0.8953323364257812 1
0.892181396484375 0.9111328125 2
0.8820648193359375 0.8846588134765625 3
0.8791961669921875 0.8802947998046875 4
0.8449630737304688 0.86090087890625 5


## supernet

In [32]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [ ]:
import pennylane as qml
import numpy as np
import os

from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from create_noise_models import noisy_dev_from_backend

dataset = 'mnist_2'
main_dir = './supernet/mnist_2/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4

# device_name = 'ibm_nairobi'
dev = qml.device('lightning.qubit', wires=num_qubits)
# dev = noisy_dev_from_backend(device_name, num_qubits)

# inds = np.random.choice(len(x_test), 400, False)
# x_test = x_test[inds]
# y_test = y_test[inds]

param_nums = [4, 8, 12, 16, 20]

for p in param_nums:
    supernet_dir = main_dir + 'search-{}_params_mb'.format(p)

    num_layers = p // num_qubits

    layer_rots = open(supernet_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(supernet_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    search_space = len(layer_rots) * len(layer_cnots)
        
    accs = []
    losses = []
    circ_layers = []

    params = np.genfromtxt(supernet_dir + '/training_params.txt')[-1].reshape((num_layers, search_space // num_cnot_configs, num_qubits))
        
#     supernet_device_dir = supernet_dir + '/' + device_name
    supernet_device_dir = supernet_dir
    
#     if not os.path.exists(supernet_device_dir):
#         os.mkdir(supernet_device_dir)
        
    for i in range(num_circs):
        curr_circ_desc = np.random.randint(0, search_space, num_layers)

#         curr_params = np.concatenate([params[k, curr_circ_desc[k] // num_cnot_configs] for k in range(num_layers)]).flatten()
        curr_params = np.zeros(num_layers * num_qubits)

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(curr_circ_desc, num_embed_layers, layer_rots, layer_cnots,
                                                                                                          num_qubits, False) 

        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, [0], 'exp')

        val_exp_list = []
        
        for j in range(len(x_test)):            
            val_exp_list.append(circ(x_test[j], curr_params))
            
        val_exps = np.array(val_exp_list).flatten()
#         val_losses = np.sum(np.power(val_exps - y_test, 2), 1)
        val_losses = np.power(val_exps - y_test, 2)
        val_loss = np.mean(val_losses)
#         acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
        acc = np.mean(val_losses < 1)
    
        print(curr_circ_desc, acc, val_loss)

        accs.append(acc)
        losses.append(val_loss)
        circ_layers.append(curr_circ_desc)
        
    np.savetxt(supernet_device_dir + '/searched_circ_layers.txt', np.array(circ_layers))
    np.savetxt(supernet_device_dir + '/searched_circ_accs.txt', accs)
    np.savetxt(supernet_device_dir + '/searched_circ_losses.txt', losses)    

[236202] 0.4825 1.1476186079715958
[299869] 0.52 1.1648819201877743
[45264] 0.4375 1.2355120238590234
[69116] 0.5525 1.1063296288359943
[110260] 0.4675 1.164864488681757
[265800] 0.5025 1.157799297365009
[301201] 0.4925 1.2290168437438584
[318918] 0.5525 1.1063296288359943
[79877] 0.5175 1.1654712539583445
[64704] 0.4375 1.2355120238590234
[107968] 0.5525 1.1063296288359943
[49761] 0.5075 1.1525933274123827
[281946] 0.5525 1.1063296288359943
[56922] 0.5025 1.157799297365009
[225702] 0.48 1.1592261433731046
[304844] 0.5025 1.157799297365009
[294116] 0.4375 1.2355120238590234
[254378] 0.48 1.1592261433731044
[162602] 0.5175 1.1654712539583445
[165081] 0.4375 1.2355120238590234
[95067] 0.52 1.1648819201877745
[151693] 0.4925 1.2290168437438584
[305188] 0.5175 1.1654712539583445
[15614] 0.4375 1.2355120238590234
[233959] 0.52 1.1648819201877743
[73226] 0.5175 1.1654712539583445
[67266] 0.5025 1.157799297365009
[17502] 0.4375 1.2355120238590234
[129295] 0.5175 1.1654712539583445
[98558] 0.4

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from train_circ import mse_loss, train_qnn
from create_noise_models import noisy_dev_from_backend

import pennylane as qml
import numpy as np
import os

dataset = 'mnist_2'
main_dir = './supernet/mnist_2/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4
meas_qubits = [0]
num_steps = 8000

dev = qml.device('lightning.qubit', wires=num_qubits)

param_nums = [20]
search_nums = [100]
num_trials = [25]

# device_name = 'ibm_nairobi'
# noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

for p in param_nums:
    curr_params_dir =  main_dir + 'search-{}_params_mb'.format(p)

    layer_rots = open(curr_params_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(curr_params_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    circ_accs = np.genfromtxt(curr_params_dir + '/{}/searched_circ_accs.txt'.format(device_name))
    circ_losses = np.genfromtxt(curr_params_dir + '/{}/searched_circ_losses.txt'.format(device_name))
    circ_layers = np.genfromtxt(curr_params_dir + '/{}/searched_circ_layers.txt'.format(device_name)).astype('int32').reshape((num_circs, p // num_qubits))

    for k in range(len(search_nums)):
        search_dir = curr_params_dir + '/{}/search_{}'.format(device_name, search_nums[k])

        if not os.path.exists(search_dir):
            os.mkdir(search_dir)

        for i in range(num_trials[k]):
            curr_dir = search_dir + '/trial_{}'.format(i + 1)

            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)

            chosen_circs = range(search_nums[k] * i, search_nums[k] * i + search_nums[k])
            best_circuit_ind = chosen_circs[np.argmax(circ_accs[chosen_circs])]
            best_subnet = circ_layers[best_circuit_ind]
            print(circ_accs[best_circuit_ind], circ_losses[best_circuit_ind], best_subnet)

            np.savetxt(curr_dir + '/sampled_circuits.txt', circ_layers[chosen_circs])
            np.savetxt(curr_dir + '/sampled_circuit_accs.txt', circ_accs[chosen_circs])

            circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(best_subnet, num_embed_layers,
                                                                                                              layer_rots, layer_cnots, num_qubits, False)

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint') 
            
            noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', None)

            losses_list = []
            accs_list = []
            
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], num_steps, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                    loss_window=50, init_params=None, 
                                                                                                    acc_thres=1.1, shuffle=True, print_loss=50)
                
                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                                
                noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                noisy_val_loss = np.array([mse_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(val_loss < 1)
                noisy_acc = np.mean(noisy_val_loss < 1)

                losses_list.append(val_loss)
                accs_list.append(acc)
                
                noisy_losses_list.append(noisy_val_loss)
                noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_dir + '/accs.txt', accs_list)
            np.savetxt(curr_dir + '/val_losses.txt', losses_list)          
            
            np.savetxt(curr_dir + '/noisy_accs.txt', noisy_accs_list)
            np.savetxt(curr_dir + '/noisy_val_losses.txt', noisy_losses_list)  

            np.savetxt(curr_dir + '/gates.txt', circ_gates, fmt="%s")
            np.savetxt(curr_dir + '/gate_params.txt', gate_params, fmt="%s")
            np.savetxt(curr_dir + '/inputs_bounds.txt', inputs_bounds)
            np.savetxt(curr_dir + '/weights_bounds.txt', weights_bounds)
            np.savetxt(curr_dir + '/layers.txt', best_subnet)

0.5825 0.9873125839233399 [313417]
[2022-10-09 05:47:00.136 tensorflow-2-3-cpu-p-ml-m5-2xlarge-47d06a51e9c5007ac68fdf260855:24 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-10-09 05:47:00.183 tensorflow-2-3-cpu-p-ml-m5-2xlarge-47d06a51e9c5007ac68fdf260855:24 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Step 1 | Sliding Loss Window : 2.050306690117393
Step 51 | Sliding Loss Window : 1.3324971478171048
Step 101 | Sliding Loss Window : 1.367552981236412
Step 151 | Sliding Loss Window : 1.0696225421971624
Step 201 | Sliding Loss Window : 1.0839896328256697
Step 251 | Sliding Loss Window : 1.4031164814374495
Step 301 | Sliding Loss Window : 1.1662724459138603
Step 351 | Sliding Loss Window : 1.2707588363735425
Step 401 | Sliding Loss Window : 1.1321641087537058
Step 451 | Sliding Loss Window : 1.1268050369270681
Step 501 | Sliding Loss Window : 1.074272805647142
Step 551 | Sliding Loss Window : 1.1

ValueError: cannot reshape array of size 10 into shape (2500,2)

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from train_circ import mse_vec_loss, train_qnn
from create_noise_models import noisy_dev_from_backend

import pennylane as qml
import numpy as np
import os

dataset = 'fmnist_4'
main_dir = './supernet/fmnist_4/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4
meas_qubits = [0, 1]
num_steps = 16000

dev = qml.device('lightning.qubit', wires=num_qubits)

param_nums = [24]
search_nums = [100]
num_trials = [25]

device_name = 'ibm_nairobi'
noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

for p in param_nums:
    curr_params_dir =  main_dir + 'search-{}_params_mb'.format(p)

    layer_rots = open(curr_params_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(curr_params_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    circ_accs = np.genfromtxt(curr_params_dir + '/{}/searched_circ_accs.txt'.format(device_name))
    circ_losses = np.genfromtxt(curr_params_dir + '/{}/searched_circ_losses.txt'.format(device_name))
    circ_layers = np.genfromtxt(curr_params_dir + '/{}/searched_circ_layers.txt'.format(device_name)).astype('int32').reshape((num_circs, p // num_qubits))

    for k in range(len(search_nums)):
        search_dir = curr_params_dir + '/{}/search_{}'.format(device_name, search_nums[k])

        if not os.path.exists(search_dir):
            os.mkdir(search_dir)

        for i in range(12):
            curr_dir = search_dir + '/trial_{}'.format(i + 1)

            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)

            chosen_circs = range(search_nums[k] * i, search_nums[k] * i + search_nums[k])
            best_circuit_ind = chosen_circs[np.argmax(circ_accs[chosen_circs])]
            best_subnet = circ_layers[best_circuit_ind]
            print(circ_accs[best_circuit_ind], circ_losses[best_circuit_ind], best_subnet)

            np.savetxt(curr_dir + '/sampled_circuits.txt', circ_layers[chosen_circs])
            np.savetxt(curr_dir + '/sampled_circuit_accs.txt', circ_accs[chosen_circs])

            circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(best_subnet, num_embed_layers,
                                                                                                              layer_rots, layer_cnots, num_qubits, False)

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint') 
            
            noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', None)

            losses_list = []
            accs_list = []
            
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], num_steps, 0.05, 1, mse_vec_loss, verbosity=17300, 
                                                                                                    loss_window=50, init_params=None, 
                                                                                                    acc_thres=1.1, shuffle=True, print_loss=50)
                
                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                                
                noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                noisy_val_loss = np.array([mse_vec_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
                noisy_acc = np.mean(np.sum(np.multiply(noisy_val_exps, y_test) > 0, 1) == 2)
#                 acc = np.mean(val_loss < 1)
#                 noisy_acc = np.mean(noisy_val_loss < 1)

                losses_list.append(val_loss)
                accs_list.append(acc)
                
                noisy_losses_list.append(noisy_val_loss)
                noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_dir + '/accs.txt', accs_list)
            np.savetxt(curr_dir + '/val_losses.txt', losses_list)          
            
            np.savetxt(curr_dir + '/noisy_accs.txt', noisy_accs_list)
            np.savetxt(curr_dir + '/noisy_val_losses.txt', noisy_losses_list)  

            np.savetxt(curr_dir + '/gates.txt', circ_gates, fmt="%s")
            np.savetxt(curr_dir + '/gate_params.txt', gate_params, fmt="%s")
            np.savetxt(curr_dir + '/inputs_bounds.txt', inputs_bounds)
            np.savetxt(curr_dir + '/weights_bounds.txt', weights_bounds)
            np.savetxt(curr_dir + '/layers.txt', best_subnet)

ibmqfactory.load_account:WARNING:2022-10-14 13:21:38,729: Credentials are already in use. The existing account in the session will be replaced.


0.3275 2.037517614364624 [ 58066 217763 135972  57519  29658 300129]
Step 1 | Sliding Loss Window : 1.4794653501819306
Step 51 | Sliding Loss Window : 1.4948715433511282
Step 101 | Sliding Loss Window : 1.3888903312489795
Step 151 | Sliding Loss Window : 1.4473596506471893
Step 201 | Sliding Loss Window : 1.4606601689456626
Step 251 | Sliding Loss Window : 1.4302624387149188
Step 301 | Sliding Loss Window : 1.414287418079292
Step 351 | Sliding Loss Window : 1.4359737427539796
Step 401 | Sliding Loss Window : 1.4009853822687786
Step 451 | Sliding Loss Window : 1.4086399233743074
Step 501 | Sliding Loss Window : 1.389459117055906
Step 551 | Sliding Loss Window : 1.351650110298885
Step 601 | Sliding Loss Window : 1.45255000125876
Step 651 | Sliding Loss Window : 1.4052908334438854
Step 701 | Sliding Loss Window : 1.4903525671417506
Step 751 | Sliding Loss Window : 1.4163437933970535
Step 801 | Sliding Loss Window : 1.3475601888329718
Step 851 | Sliding Loss Window : 1.4041627675447053
Ste

## train correlation cricuits

In [ ]:
from datasets import TorchDataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss

import pickle as pkl
import torch
import os
import numpy as np

dataset = 'fmnist_2'
curr_dir = f'./experiment_data/{dataset}/trained_circuits/'

num_qubits = 4
num_embeds = 16

num_train_steps = 2500
num_test_data = 400

num_meas_qubits = 1

loss = TQMseLoss(num_meas_qubits, 4)

dev = qml.device('lightning.qubit', wires=4)
device = 'cpu'

train_data = TorchDataset(dataset, 'angle', 1, reshape_labels=True)
test_data = TorchDataset(dataset, 'angle', 1, False, reshape_labels=True)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for i in range(400, 600):    
    circ_dir = curr_dir + f'circ_{i + 1}'
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    losses_list = []
    accs_list = []
    
    for j in range(5):
        curr_train_dir = circ_dir + '/run_{}'.format(j + 1)
        
        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
        opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
        curr_loss, curr_acc = train_tq_model(model, num_meas_qubits, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)
        
        print(curr_loss, curr_acc)
        
        torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        losses_list.append(curr_loss)
        accs_list.append(curr_acc)
        
    np.savetxt(circ_dir + '/val_losses.txt', losses_list)
    np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 2.208944320678711
Step 101 | Loss: 0.147115096449852
Step 201 | Loss: 0.2950589954853058
Step 301 | Loss: 0.26141610741615295
Step 401 | Loss: 0.3533658981323242
Step 501 | Loss: 0.32431575655937195
Step 601 | Loss: 0.10195403546094894
Step 701 | Loss: 0.20054180920124054
Step 801 | Loss: 0.10636462271213531
Step 901 | Loss: 0.10576574504375458
Step 1001 | Loss: 0.12485537678003311
Step 1101 | Loss: 0.20588746666908264
Step 1201 | Loss: 0.14109914004802704
Step 1301 | Loss: 0.03189925476908684
Step 1401 | Loss: 0.1251666098833084
Step 1501 | Loss: 0.2581541836261749
Step 1601 | Loss: 0.06814759224653244
Step 1701 | Loss: 0.15024860203266144
Step 1801 | Loss: 0.10150475800037384
Step 1901 | Loss: 0.2396751344203949
Step 2001 | Loss: 0.38675063848495483
Step 2101 | Loss: 0.13071303069591522
Step 2201 | Loss: 0.2848568558692932
Step 2301 | Loss: 0.11898909509181976
Step 2401 | Loss: 0.13577520847320557
0.18491865514193048 0.9425
Step 1 | Loss: 1.2461879253387451
Step 101 | 

In [ ]:
from datasets import TorchDataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss

import pickle as pkl
import torch
import os
import numpy as np

dataset = 'vowel_4'
curr_dir = f'./experiment_data/{dataset}/trained_circuits/'

num_qubits = 4
num_embeds = 10

num_train_steps = 1000
num_test_data = 120

num_meas_qubits = 2

loss = TQMseLoss(num_meas_qubits, 4)

dev = qml.device('lightning.qubit', wires=4)
device = 'cpu'

train_data = TorchDataset(dataset, 'angle', 1, reshape_labels=False)
test_data = TorchDataset(dataset, 'angle', 1, False, reshape_labels=False)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for i in range(200):    
    circ_dir = curr_dir + f'circ_{i + 1}'
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    losses_list = []
    accs_list = []
    
    for j in range(5):
        curr_train_dir = circ_dir + '/run_{}'.format(j + 1)
        
        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
        opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
        curr_loss, curr_acc = train_tq_model(model, num_meas_qubits, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)
        
        print(curr_loss, curr_acc)
        
        torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        losses_list.append(curr_loss)
        accs_list.append(curr_acc)
        
    np.savetxt(circ_dir + '/val_losses.txt', losses_list)
    np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 2.542893409729004
Step 101 | Loss: 1.959916114807129
Step 201 | Loss: 1.7214940786361694
Step 301 | Loss: 1.9240800142288208
Step 401 | Loss: 1.869226098060608
Step 501 | Loss: 1.6790121793746948
Step 601 | Loss: 1.6700599193572998
Step 701 | Loss: 1.8174173831939697
Step 801 | Loss: 1.856374740600586
Step 901 | Loss: 1.5111029148101807
1.7113907750473651 0.45
Step 1 | Loss: 2.2041118144989014
Step 101 | Loss: 1.87391996383667
Step 201 | Loss: 1.8433352708816528
Step 301 | Loss: 1.8049499988555908
Step 401 | Loss: 1.694274663925171
Step 501 | Loss: 1.6473220586776733
Step 601 | Loss: 1.551181674003601
Step 701 | Loss: 1.7548195123672485
Step 801 | Loss: 1.750704050064087
Step 901 | Loss: 1.6355087757110596
1.6943052350523353 0.4
Step 1 | Loss: 1.9824429750442505
Step 101 | Loss: 2.0142054557800293
Step 201 | Loss: 2.000779151916504
Step 301 | Loss: 1.9163753986358643
Step 401 | Loss: 1.6260305643081665
Step 501 | Loss: 1.7207516431808472
Step 601 | Loss: 1.82187509536743

## human design circs

In [93]:
from datasets import TorchDataset
from datasets_nt import load_dataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss
from create_noise_models import noisy_dev_from_backend
from create_human_design_circs import convert_human_design_circ_to_gate_circ

import pickle as pkl
import torch
import os
import numpy as np

embed_method = 'iqp'

dataset = 'fmnist_2'
curr_dir = f'./human_design/{embed_method}_basic/{dataset}'

num_qubits = 4
num_embed_layers = 4
num_dataset_reps = 1

num_train_steps = 2500
num_test_data = 400

num_meas_qubits = 1

param_nums = [16, 20, 24, 28, 32]

if not os.path.exists(curr_dir):
    os.mkdir(curr_dir)

loss = TQMseLoss(num_meas_qubits, 4)
device = 'cpu'

train_data = TorchDataset(dataset, embed_method, num_dataset_reps, reshape_labels=True)
test_data = TorchDataset(dataset, embed_method, num_dataset_reps, False, reshape_labels=True)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for param_num in param_nums:
    curr_param_dir = curr_dir + f'/{param_num}_params/'
    
    if not os.path.exists(curr_param_dir):
        os.mkdir(curr_param_dir)
    
    for i in range(1):    
        circ_dir = curr_param_dir

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_human_design_circ_to_gate_circ(num_qubits, embed_method, 'basic', num_embed_layers,
                                                                                                        param_num // num_qubits)
        
        np.savetxt(curr_param_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_param_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_param_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_param_dir + '/weights_bounds.txt', weights_bounds)
        
        losses_list = []
        accs_list = []

        for j in range(5):
            curr_train_dir = circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)

            model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
            opt = torch.optim.SGD(model.parameters(), lr=0.05)

            curr_loss, curr_acc = train_tq_model(model, num_meas_qubits, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)

            print(curr_loss, curr_acc)

            losses_list.append(curr_loss)
            accs_list.append(curr_acc)
            
            torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        np.savetxt(circ_dir + '/val_losses.txt', losses_list)
        np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 1.0241748094558716
Step 101 | Loss: 0.8487192988395691
Step 201 | Loss: 0.7471197843551636
Step 301 | Loss: 0.6941156983375549
Step 401 | Loss: 0.9297991991043091
Step 501 | Loss: 0.8871689438819885
Step 601 | Loss: 0.791683554649353
Step 701 | Loss: 0.7008715867996216
Step 801 | Loss: 0.7160510420799255
Step 901 | Loss: 0.8127270340919495
Step 1001 | Loss: 0.8396831750869751
Step 1101 | Loss: 0.8218494057655334
Step 1201 | Loss: 0.6978917121887207
Step 1301 | Loss: 0.7268744707107544
Step 1401 | Loss: 0.6382741928100586
Step 1501 | Loss: 0.7067519426345825
Step 1601 | Loss: 0.7633805274963379
Step 1701 | Loss: 0.8198347687721252
Step 1801 | Loss: 0.7686297297477722
Step 1901 | Loss: 0.7931156754493713
Step 2001 | Loss: 0.6907697916030884
Step 2101 | Loss: 0.8034773468971252
Step 2201 | Loss: 0.6209926009178162
Step 2301 | Loss: 0.7236651182174683
Step 2401 | Loss: 0.7283642292022705
0.7098884667784855 0.8175
Step 1 | Loss: 1.0437079668045044
Step 101 | Loss: 0.841492295

In [92]:
from importlib import reload

import datasets_nt
import datasets
import create_gate_circs_np

reload(datasets_nt)
reload(datasets)
reload(create_gate_circs_np)

<module 'create_gate_circs_np' from '/root/create_gate_circs_np.py'>